In [ ]:
import cv2
import urllib.request
import numpy as np
from IPython.display import display, clear_output
from PIL import Image
from io import BytesIO
from math import sqrt
import time

In [ ]:
# when you type the url into the web browser, it shows a single image.
# This code sends continous requests to the url , allowing it to continuously get frames

class StreamCapture:
    def __init__(self):
        self.ip = input("ESP cam IP: ").strip()
        self.res = input("(L)ow, (M)edium, or (H)igh Resolution: ").strip().upper()
        self.route = ""

    def process(self):
        if(self.res == "H"):
            self.route = "cam-hi.jpg"
        elif(self.res == "M"):
            self.route = "cam-mid.jpg"
        else:
            self.route = "cam-lo.jpg"

        url = f'http://{self.ip}/{self.route}'

        start_time = time.time()
        frame_count = 0

        # Read and display video frames
        while True:
            try:
                # Read a frame from the video stream
                img_resp=urllib.request.urlopen(url)
                imgnp=cv2.imdecode(np.array(bytearray(img_resp.read()),dtype=np.uint8),  cv2.IMREAD_COLOR)
                frame_count += 1                

                current_time = time.time()
                elapsed_time = current_time-start_time
                fps = frame_count / elapsed_time if elapsed_time > 0 else 0

                process_frame(imgnp, fps)

            except Exception as e:
                print(f"ERROR: {e}")
                exit()
            

In [ ]:
def detect_color(frame, color_ranges, color_name):
    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    mask = None
    for color_range in color_ranges:
        # Create a mask for each color range
        current_mask = cv2.inRange(hsv, color_range[0], color_range[1])
        if mask is None:
            mask = current_mask
        else:
            mask = cv2.bitwise_or(mask, current_mask)
    
    # Apply morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, mask, color_name

In [ ]:
def calculate_average_color(contour, frame):
    mask = np.zeros(frame.shape[:2], dtype="uint8")
    cv2.drawContours(mask, [contour], -1, 255, -1)
    mean_val = cv2.mean(frame, mask=mask)
    return mean_val[:3]  # Return BGR values

In [ ]:
def process_frame(frame, fps):

    red_ranges = [
            (np.array([0, 120, 70]), np.array([10, 255, 255])),
            (np.array([160, 120, 70]), np.array([180, 255, 255]))
        ]
    green_range = [
        (np.array([35, 50, 100], dtype=np.uint8), np.array([85, 255, 255], dtype=np.uint8))]
    
    # Detect colors
    red_contours, _, _ = detect_color(frame, red_ranges, "Red")
    green_contours, _, _ = detect_color(frame, green_range, "Green")
    
    # Process contours
    for contours, color_name in [(red_contours, "Red"), (green_contours, "Green")]:
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 200:
                x, y, w, h = cv2.boundingRect(cnt)
                average_color = calculate_average_color(cnt, frame)
                color_text = f"R:{int(average_color[2])}, G:{int(average_color[1])}, B:{int(average_color[0])}"
                box_color = (0, 0, 0)
                if(color_name == "Red"):
                    box_color = (0, 0, 255)
                else:
                    box_color = (0, 255 ,0)
                frame = cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
                cv2.putText(frame, f"{color_name}: {color_text}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display FPS on the frame
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Convert and display the frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    display_image = Image.fromarray(frame_rgb)
    display(display_image)
    clear_output(wait=True)

In [ ]:
if __name__ == "__main__":
    print("Hello World!")
    streamcap = StreamCapture()
    streamcap.process()